In [1]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import date, timedelta
import calendar

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()
engine = create_engine("sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3")
conlt = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
year = 2022
quarter = 3
today = date.today()
yesterday = today - timedelta(days=1)
today, yesterday

(datetime.date(2022, 11, 30), datetime.date(2022, 11, 29))

In [2]:
#yesterday = today - timedelta(days=3)
yesterday

datetime.date(2022, 11, 29)

In [3]:
format_dict = {    
    'shares':'{:,}',    
    'q4':'{:.4f}','q3':'{:.4f}','q2':'{:.4f}','q1':'{:.4f}','dividend':'{:.4f}','qtrly':'{:.4f}',
    'price':'{:.2f}',    
    'amount':'{:,.2f}','net':'{:,.2f}','cost_amt':'{:,.2f}',
    'yield':'{:,.2f}%','pct':'{:,.2f}%',
    'xdate':'{:%Y-%m-%d}','paiddate':'{:%Y-%m-%d}',
              }

### New dividend stock

In [4]:
name = 'LHK'
sql = """
SELECT * 
FROM DIVIDEND
WHERE name = '%s'
"""
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual


In [5]:
sqlIns = """
INSERT INTO dividend
VALUES('LHK',0,0.20,0,0.45,0.65,0,0,00,'2022-11-25','2022-12-09','',1)
"""
rp = const.execute(sqlIns)
rp.rowcount

1

### Start of Update dividend

In [4]:
def update_dividend1(q3,XDATE,PAIDDATE,actual,name):
    sql = "UPDATE dividend SET q3 = %s, dividend = q1+q2+q3+q4, XDATE = '%s', PAIDDATE = '%s', actual = %s WHERE name = '%s'"
    sql = sql % (q3,XDATE, PAIDDATE, actual,name)
    rp = const.execute(sql)
    return "Records updated = " + str(rp.rowcount)

In [7]:
name = 'TFFIF'
sql = """
SELECT * FROM DIVIDEND WHERE name = '%s'"""
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,TFFIF,0.0928,0.1003,0.0920,0.0870,0.3721,0,2022-12-09,2022-12-28,,1


In [6]:
q3 = 0.1003
XDATE = '2022-12-09'
PAIDDATE = '2022-12-28'
actual = 1

update_dividend1(q3,XDATE,PAIDDATE,actual,name)

'Records updated = 1'

In [4]:
def update_dividend2(shares,q3,actual,name):
    sql = "UPDATE dividend SET shares = %s, q3 = %s, dividend=q1+q2+q3+q4, actual = %s WHERE name = '%s'"
    sql = sql % (shares, q3, actual, name)
    rp = const.execute(sql)
    return "Records updated = " + str(rp.rowcount)

In [5]:
name = 'STA'
sql = """
SELECT * 
FROM DIVIDEND
WHERE name = '%s'
"""
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,STA,0.6500,1.2500,1.2500,0.0000,3.1500,"20,000",2022-04-19,2022-05-06,,0


In [6]:
shares = 20000
q3 = 0
actual = 1

In [7]:
update_dividend2(shares,q3,actual,name)

'Records updated = 1'

### Toggle actual status

In [6]:
name = 'EGATIF'
actual = 0
sqlUpd = "UPDATE dividend SET actual = %s WHERE name = '%s'"
sqlUpd = sqlUpd % (actual, name)
rp = const.execute(sqlUpd)
rp.rowcount

1

### Delete dividend record

In [10]:
sqlDel = '''
DELETE FROM dividend
WHERE name IN ("MC")
'''
rp = const.execute(sqlDel)
rp.rowcount

1

### Start of output to cloud

In [4]:
sql = """
SELECT Y.NAME AS name,  Q3 AS qtrly, SHARES, XDATE, PAIDDATE, 
P.price AS price, Y.DIVIDEND, ACTUAL, B.price * B.volbuy AS cost_amt
FROM dividend AS Y, price AS P, buy AS B
WHERE Y.name = P.name 
AND Y.name = B.name 
AND Q3 > 0 
AND P.date = '%s'
ORDER BY name
"""
sql = sql % yesterday 
print(sql)


SELECT Y.NAME AS name,  Q3 AS qtrly, SHARES, XDATE, PAIDDATE, 
P.price AS price, Y.DIVIDEND, ACTUAL, B.price * B.volbuy AS cost_amt
FROM dividend AS Y, price AS P, buy AS B
WHERE Y.name = P.name 
AND Y.name = B.name 
AND Q3 > 0 
AND P.date = '2022-09-30'
ORDER BY name



In [5]:
cols = 'name qtrly shares amount net xdate paiddate cost_amt pct actual'.split()

In [6]:
dividend = pd.read_sql(sql, const)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend['amount'] = round(dividend['shares'] * dividend['qtrly'], 2)
dividend['net'] = round(dividend['amount'] * 0.9, 2)
dividend['pct'] = round(dividend['net'] / dividend['cost_amt'] * 100, 2)
dividend[cols].style.format(format_dict)

,name,qtrly,shares,amount,net,xdate,paiddate,cost_amt,pct,actual
0,CPNREIT,0.1814,"10,000","1,814.00","1,632.60",2022-08-17,2022-09-02,"192,000.00",0.85%,0
1,DCC,0.0400,"60,000","2,400.00","2,160.00",2022-08-22,2022-09-06,"177,600.00",1.22%,0
2,DIF,0.2610,"40,000","10,440.00","9,396.00",2022-08-16,2022-09-05,"588,000.00",1.60%,0
3,GVREIT,0.1765,"10,000","1,765.00","1,588.50",2022-08-26,2022-09-14,"91,500.00",1.74%,0
4,IVL,0.2500,"2,400",600.00,540.00,2022-08-30,2022-09-15,"105,600.00",0.51%,0
5,JASIF,0.2200,"130,000","28,600.00","25,740.00",2022-08-19,2022-09-06,"1,300,000.00",1.98%,0
6,MCS,0.3700,"75,000","27,750.00","24,975.00",2022-09-03,2022-09-15,"1,155,000.00",2.16%,0
7,ORI,0.1000,"45,000","4,500.00","4,050.00",2022-08-26,2022-09-13,"495,000.00",0.82%,0
8,RCL,1.5000,"15,000","22,500.00","20,250.00",2022-08-18,2022-09-02,"686,250.00",2.95%,0
9,WHAIR,0.1738,"40,000","6,952.00","6,256.80",2022-08-18,2022-09-19,"358,000.00",1.75%,0


In [7]:
dividend.net.sum()

103482.00000000001

In [9]:
file_name = 'dividend-q3.xlsx'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

dividend[cols].sort_values(['name'],ascending=[True]).to_excel(output_file, index=False)
dividend[cols].sort_values(['name'],ascending=[True]).to_excel(data_file, index=False)
dividend[cols].sort_values(['name'],ascending=[True]).to_excel(box_file, index=False)
dividend[cols].sort_values(['name'],ascending=[True]).to_excel(one_file, index=False)

### End of output to cloud

In [14]:
sql = 'SELECT name, q_eps, aq_eps, publish_date FROM epss WHERE year = %s-1 AND quarter = %s ORDER BY name'
sql = sql % (year, quarter)
print(sql)
eps = pd.read_sql(sql, conlt)
eps.dtypes

SELECT name, q_eps, aq_eps, publish_date FROM epss WHERE year = 2022-1 AND quarter = 3 ORDER BY name


name             object
q_eps           float64
aq_eps          float64
publish_date     object
dtype: object

In [15]:
df_merge = pd.merge(dividend, eps, on='name', how='inner')
df_merge.sort_values('name',ascending=True)

,name,qtrly,shares,xdate,paiddate,price,dividend,actual,cost_amt,amount,net,pct,q_eps,aq_eps,publish_date
0,CPNREIT,0.1814,10000,2022-08-17,2022-09-02,18.60,0.7873,0,192000.0,1814.0,1632.6,0.85,-0.3294,-0.1099,2021-11-08
1,DCC,0.0400,60000,2022-08-22,2022-09-06,2.68,0.1750,0,177600.0,2400.0,2160.0,1.22,0.0400,0.1510,2021-11-08
2,DIF,0.2610,40000,2022-08-16,2022-09-05,13.50,1.0420,0,588000.0,10440.0,9396.0,1.60,0.2769,0.8319,2021-11-12
3,GVREIT,0.1765,10000,2022-08-26,2022-09-14,9.00,0.7640,0,91500.0,1765.0,1588.5,1.74,0.2459,0.7621,2021-08-09
4,IVL,0.2500,2400,2022-08-30,2022-09-15,39.00,1.3000,0,105600.0,600.0,540.0,0.51,1.1300,3.6200,2021-11-05
5,JASIF,0.2200,130000,2022-08-19,2022-09-06,8.15,0.9300,0,1300000.0,28600.0,25740.0,1.98,0.2093,0.6909,2021-11-09
6,MCS,0.3700,75000,2022-09-03,2022-09-15,10.10,0.8700,0,1155000.0,27750.0,24975.0,2.16,0.8900,2.2300,2021-11-15
7,ORI,0.1000,45000,2022-08-26,2022-09-13,10.10,0.6700,0,495000.0,4500.0,4050.0,0.82,0.2891,0.9728,2021-11-12
8,RCL,1.5000,15000,2022-08-18,2022-09-02,29.00,6.2500,0,686250.0,22500.0,20250.0,2.95,4.5020,11.9000,2021-11-05
9,WHAIR,0.1738,40000,2022-08-18,2022-09-19,7.25,0.6588,0,358000.0,6952.0,6256.8,1.75,0.1684,0.6664,2021-11-11


In [16]:
colt = 'name qtrly shares amount net publish_date xdate paiddate q_eps aq_eps'.split()

In [17]:
df_merge[colt].sort_values('publish_date',ascending=True)

,name,qtrly,shares,amount,net,publish_date,xdate,paiddate,q_eps,aq_eps
3,GVREIT,0.1765,10000,1765.0,1588.5,2021-08-09,2022-08-26,2022-09-14,0.2459,0.7621
4,IVL,0.2500,2400,600.0,540.0,2021-11-05,2022-08-30,2022-09-15,1.1300,3.6200
8,RCL,1.5000,15000,22500.0,20250.0,2021-11-05,2022-08-18,2022-09-02,4.5020,11.9000
0,CPNREIT,0.1814,10000,1814.0,1632.6,2021-11-08,2022-08-17,2022-09-02,-0.3294,-0.1099
1,DCC,0.0400,60000,2400.0,2160.0,2021-11-08,2022-08-22,2022-09-06,0.0400,0.1510
5,JASIF,0.2200,130000,28600.0,25740.0,2021-11-09,2022-08-19,2022-09-06,0.2093,0.6909
10,WHART,0.2553,30000,7659.0,6893.1,2021-11-10,2022-08-18,2022-09-02,0.1796,0.5478
9,WHAIR,0.1738,40000,6952.0,6256.8,2021-11-11,2022-08-18,2022-09-19,0.1684,0.6664
2,DIF,0.2610,40000,10440.0,9396.0,2021-11-12,2022-08-16,2022-09-05,0.2769,0.8319
7,ORI,0.1000,45000,4500.0,4050.0,2021-11-12,2022-08-26,2022-09-13,0.2891,0.9728


In [18]:
file_name = 'div-22q3.xlsx'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_merge[colt].sort_values(['name'],ascending=[True]).to_excel(output_file, index=False)
df_merge[colt].sort_values(['name'],ascending=[True]).to_excel(data_file, index=False)
df_merge[colt].sort_values(['name'],ascending=[True]).to_excel(box_file, index=False)
df_merge[colt].sort_values(['name'],ascending=[True]).to_excel(one_file, index=False)